# ASL Letter Recognition

Dependencies  
`pip install tensorflow`
`pip install tensorflow-object-detection-api`

In [1]:
import os

## Setup

In [2]:
WORKPLACE_PATH = 'Tensorflow/workplace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKPLACE_PATH + '/annotations'
IMAGE_PATH = WORKPLACE_PATH + '/images'
MODEL_PATH = WORKPLACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKPLACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

## Generate Labels

In [3]:
labels = [{'name':'a', 'id':1},
          {'name':'b', 'id':2},
          {'name':'c', 'id':3},
          {'name':'d', 'id':4},
          {'name':'e', 'id':5},
          {'name':'f', 'id':6},
          {'name':'g', 'id':7},
          {'name':'h', 'id':8},
          {'name':'i', 'id':9},
          {'name':'j', 'id':10},
          {'name':'k', 'id':11},
          {'name':'l', 'id':12},
          {'name':'m', 'id':13},
          {'name':'n', 'id':14},
          {'name':'o', 'id':15},
          {'name':'p', 'id':16},
          {'name':'q', 'id':17},
          {'name':'r', 'id':18},
          {'name':'s', 'id':19},
          {'name':'t', 'id':20},
          {'name':'u', 'id':21},
          {'name':'v', 'id':22},
          {'name':'w', 'id':23},
          {'name':'x', 'id':24},
          {'name':'y', 'id':25},
          {'name':'z', 'id':26},
          ]

In [ ]:
with open(os.path.join(ANNOTATION_PATH, 'label_map.pbtxt'), 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Generate TF records

In [ ]:
if os.name == 'posix':
    !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
    !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}
else:
    !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
    !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

## Download pretrained TF models

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

In [ ]:
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

In [4]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [ ]:
!mkdir {MODEL_PATH + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

In [5]:
import tensorflow as tf
# from object_detection.utils import config_util
# from object_detection.protos import pipeline_pb2
# from google.protobuf import text_format

model.ssd.num_classes = 26

train_config.batch_size = 13

train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

train_config.fine_tune_checkpoint_type = "detection"

train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'

train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']

eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'

eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [13]:
print("--- for posix ---")
print(f"""python3 {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps=3000""")

print("\n--- else ---")
print(f"""python {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps=3000""")

--- for posix ---
python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workplace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workplace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000

--- else ---
python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workplace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workplace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000
